In [1]:
from all_imports import *

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
PATH = Path('/scratch/arka/Ark_git_files/ecg_data/')
BPATH = PATH / 'beats_data'

In [ ]:
class PTBData(ImageClassifierData):